In [1]:
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True
)

10.9%

100.0%


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100.0%
63.6%

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100.0%


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100.0%

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [7]:

train_loader = DataLoader(train_data, batch_size=100, shuffle=True, num_workers=1)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), kernel_size=2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)

        return F.softmax(x)

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # I don't have gpu :(

model = Net().to(device)

optimizer = optim.Adam(model.parameters(), lr = 0.001)

loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    model.train()

    for batch_index, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        output = model(data)
        loss = loss_fn(output, target)
        loss.backward() # backpropogate, store gradients in tensors, then optimize with optimizer.step()
        optimizer.step()

        if batch_index % 20 == 0:
            print(f'Train Epohc: {epoch} [{batch_index * len(data)} / {len(train_loader.dataset)} ({100. * batch_index / len(train_loader):.0f}%)]\t{loss.item():.6f}')

In [9]:
for epoch in range(1, 11):
    # Training phase
    train(epoch)

/var/folders/ls/cp2_dyh951b1_cpsnk6sgxz40000gn/T/ipykernel_82703/2461379147.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Train Epohc: 1 [0 / 60000 (0%)]	2.300626
Train Epohc: 1 [2000 / 60000 (3%)]	2.284684
Train Epohc: 1 [4000 / 60000 (7%)]	2.225963
Train Epohc: 1 [6000 / 60000 (10%)]	2.080429
Train Epohc: 1 [8000 / 60000 (13%)]	1.960744
Train Epohc: 1 [10000 / 60000 (17%)]	1.872799
Train Epohc: 1 [12000 / 60000 (20%)]	1.798654
Train Epohc: 1 [14000 / 60000 (23%)]	1.781282
Train Epohc: 1 [16000 / 60000 (27%)]	1.776863
Train Epohc: 1 [18000 / 60000 (30%)]	1.764244
Train Epohc: 1 [20000 / 60000 (33%)]	1.690075
Train Epohc: 1 [22000 / 60000 (37%)]	1.729305
Train Epohc: 1 [24000 / 60000 (40%)]	1.707640
Train Epohc: 1 [26000 / 60000 (43%)]	1.685181
Train Epohc: 1 [28000 / 60000 (47%)]	1.753054
Train Epohc: 1 [30000 / 60000 (50%)]	1.703810
Train Epohc: 1 [32000 / 60000 (53%)]	1.739537
Train Epohc: 1 [34000 / 60000 (57%)]	1.645562
Train Epohc: 1 [36000 / 60000 (60%)]	1.642065
Train Epohc: 1 [38000 / 60000 (63%)]	1.719040
Train Epohc: 1 [40000 / 60000 (67%)]	1.641176
Train Epohc: 1 [42000 / 60000 (70%)]	1.619414

In [10]:
# Save the trained model weights to a file
torch.save(model.state_dict(), 'model_weights.pth')